In [1]:
import json
import jsonlines
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
dir_path = "../Data/annotations/"
ano = "open_images_train_v6_captions.jsonl"

In [15]:
image_id = []
captions = []
with jsonlines.open(dir_path+ano, 'r') as f:
    for line in f.iter():
        #if line['image_id'][0] == '0':
        image_id.append(line['image_id'])
        captions.append(line['caption'])


In [16]:
annotations = pd.DataFrame(image_id, columns=['image_id'])
annotations['caption'] = captions

In [24]:
annotations

,image_id,caption
0,fc06c1b59dd8ca6e,"We can see dolls,book and papers on bed and we..."
1,5833b3358fd949b5,In this picture I can see there is a airplane ...
2,67f1e4231b964a23,This looks like a black and white poster. I ca...
3,4839f5eac98771bf,In this image I can see on the left side it is...
4,64373c6a90e7b8dd,In this picture I can see group of people sitt...
...,...,...
507439,8cf26e4563f8c4bc,There is a lady wearing a tag and holding the ...
507440,c960dd40130f1d9a,In the image we can see there is a newspaper c...
507441,d4c2172f7f742d60,In the picture we can see table with drawers. ...
507442,60b2df9a3a7bd8d2,In this image we can see a girl. In the back t...


In [27]:
path = '../../disk/Data/open_image/'

img_paths = []
img_names = []
for folder, subfolders, filenames in os.walk(path):
    for img in filenames:
        img_paths.append(folder+'/'+img)
        img_names.append(img)
            
print('Images: ',len(img_names))


Images:  1743042


In [10]:
print(img_names[:5])

['0d602fc924a0a2a5.jpg', '0d5ad6dead4fd8d4.jpg', '0061e0fb55ff2d5c.jpg', '0abf55bf669af9c6.jpg', '0383b6b35606ce7e.jpg']


In [63]:
keyword = ['tsunami',
           'tidal wave',
           'hail',
           'hailstones',
        ]
exclude_keyword = ['blazer','horse','candle',
                   'fireplace',
                   'lamp',
                   'photograph',
                   'ski','board','says','oven','cook', 'living', 'plug', 'hyr', 'food', 'young','kitchen', 'extinguisher',
                 'fireworks','vintage','department', 'kid', 'boy', 'grill','fire station','fox','engine', 'hyd','fires','jet','rocket']
k_image_id = []

In [19]:
annotations[annotations['caption'].str.contains('fire', case=False)]

,image_id,caption
8,40c66d14998ad13a,In this image I can see a woman wearing red co...
56,7164732223313801,This is a person standing. This looks like a f...
558,faec0d8ce842e27e,"This is an animated image , where there is a b..."
826,de51ffcf704a22ab,In this image we can see group of vehicles par...
1025,2ad42a74006edb6f,"In this picture I can see sculptures, there is..."
...,...,...
506198,70c61f3f25c634b8,"In this image, I can see few people standing a..."
506239,3f232d160c0d2f5d,There are three people standing. This person i...
506493,5380decd21c84b97,In this image there is a tent supported with m...
506789,6c03acbd11f6e2be,In this picture we can see a room with pillows...


In [64]:
find = annotations[annotations['caption'].str.contains('dododo', case=False)]

for i in keyword:
    find2 = annotations[annotations['caption'].str.contains(i, case=False)]
    find = pd.concat([find,find2], ignore_index=True)
    print(f'keyword : [{i}] // found: {len(find2)}')
print(f'final : {len(find)}')
## 결측지 유발 키워드 삭제
for i in exclude_keyword:
    origin_len = len(find)
    find = find[~find['caption'].str.contains(i, case=False)]
    print(f'excluded keyword:[{i}] found: {origin_len - len(find)}')
print(f'final : {len(find)}')
## 중복 이미지 삭제 (한 이미지에 대한 여러 caption)
find.drop_duplicates(['image_id'], inplace=True ,ignore_index = True, keep = 'first')
print(f'droped duplicate images from multiple captions, After dropped: {len(find)}')
## 인덱스 정렬
find.reset_index(inplace=True, drop=True)
#find

keyword : [tsunami] // found: 0
keyword : [tidal wave] // found: 0
keyword : [hail] // found: 0
keyword : [hailstones] // found: 0
final : 0
excluded keyword:[blazer] found: 0
excluded keyword:[horse] found: 0
excluded keyword:[candle] found: 0
excluded keyword:[fireplace] found: 0
excluded keyword:[lamp] found: 0
excluded keyword:[photograph] found: 0
excluded keyword:[ski] found: 0
excluded keyword:[board] found: 0
excluded keyword:[says] found: 0
excluded keyword:[oven] found: 0
excluded keyword:[cook] found: 0
excluded keyword:[living] found: 0
excluded keyword:[plug] found: 0
excluded keyword:[hyr] found: 0
excluded keyword:[food] found: 0
excluded keyword:[young] found: 0
excluded keyword:[kitchen] found: 0
excluded keyword:[extinguisher] found: 0
excluded keyword:[fireworks] found: 0
excluded keyword:[vintage] found: 0
excluded keyword:[department] found: 0
excluded keyword:[kid] found: 0
excluded keyword:[boy] found: 0
excluded keyword:[grill] found: 0
excluded keyword:[fire st

In [65]:
find

,image_id,caption


In [66]:
find.sort_values(by=['image_id'],ascending=True, inplace=True)
find.reset_index(drop=True, inplace=True)
tmp = find['image_id'].value_counts().index.sort_values()

In [67]:
cnt = 4
for i,key in enumerate(tmp):
    if cnt == 4:
        print('', end='\n\n\n\n')
        print(f'{(i+1)/4}.')
        cnt = 0
    print(f"'{key}'", end=' ')
    cnt+=1


In [68]:
## show img status

back = 0
plt.figure(figsize=(16,500))
for i in range(len(find[back:])):
    plt.subplot(100,4,i+1)
    if i % 4 == 0:
        plt.title(f"{(i+1+back)/4}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    f_num = find['image_id'][i+back][0]
    
    path = '../../disk/Data/open_image/train_'+f_num+"/" + find['image_id'][i+back] +'.jpg'
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    plt.imshow(im_rgb, cmap=plt.cm.binary)
    plt.xlabel(find['caption'][i+back][:35] + '\n' + find['caption'][i+back][35:70] + '\n' 
               + find['caption'][i+back][70:105] + '\n image_id : ' + str(find['image_id'][i+back]) , loc='left', fontsize=10)
    #plt.legend(find['image_id'][i])
plt.show()

<Figure size 1600x50000 with 0 Axes>

In [113]:
disaster_dataset = ['005526753c3d4f8e',
                   '0177c759a586204e',
                   '0218f76dff06752d',
                   '0693141ad6ca7df3',
                   '0813556ae840d075',
                    '0bdfcf59dd6b55ce',
                   '0ff480920c9928c5','022b54e4e857ed33', '0a7e9cb174aa297d', '0de39b3791f2baa6',
                    '0023b0e23b369b8d', '009c5fb34590f1ca', '0604cb9341631e15','0737d240e1558c1b',
                    '00d425f0986ac579','0128105921974df0','01c4f65ea29a5632','05b3130e553c6084','0725cc4abfe58921'
                    
                   ]

##### 일반데이터셋 삭제

In [115]:
# path = '../Data/train_0/'
# img_names = []
# for  folder, subfolders, filenames in os.walk(path):
#     for img in filenames:
#         if img[:-4] in disaster_dataset:
#             img_names.append(img)
#         else:
#             os.remove(path + img)
            
# print('Images: ',len(img_names))
# print(img_names)

Images:  14
['0023b0e23b369b8d.jpg', '0a7e9cb174aa297d.jpg', '0693141ad6ca7df3.jpg', '0813556ae840d075.jpg', '022b54e4e857ed33.jpg', '0ff480920c9928c5.jpg', '0218f76dff06752d.jpg', '005526753c3d4f8e.jpg', '0177c759a586204e.jpg', '009c5fb34590f1ca.jpg', '0bdfcf59dd6b55ce.jpg', '0604cb9341631e15.jpg', '0de39b3791f2baa6.jpg', '0737d240e1558c1b.jpg']


In [116]:
for  folder, subfolders, filenames in os.walk(path):
    for img in filenames:
         img_names.append(img)
print('Images: ',len(img_names))

Images:  28


In [125]:
captions = []
for img_id in disaster_dataset:
    cap = annotations[annotations['image_id'] == img_id]['caption'].value_counts().index
    for j in cap:
        captions.append(j.rstrip('\n').replace(",", ""))
print(captions)

['This is an animated image. I can see three people lying on the floor and two people walking. There are some machines objects and fire.', 'In the center of the image we can see fire and some sticks. In the background of the image we can see a grass. At the top left corner we can see a person is holding bottle.', 'In the picture I can see two men are standing on the ground. These men are wearing helmets and holdings objects. In the background I can see trees the fire the grass plants the sky and some other objects on the ground.', 'In this image there are so many trees on the mountain at the back there is some fire and smoke.', 'In this image we can see people fence fire pole light and buildings. There is a dark background.', 'In this image I can see a ladder which is in red color. Background I can see a house with fire and a building in white and brown color trees in green color and sky in white color.', 'In the center of the image we can see a person walking on the ground. We can als

In [126]:
print(len(captions))

15
